In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pandas as pd
import random

from Trainer import Trainer, initialize_resnet_model

In [3]:
def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

set_seed(0)

In [4]:
# device - cpu or gpu?
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
class ContrastiveCenterLoss(nn.Module):
    def __init__(self, dim_hidden, num_classes, device, lambda_c=1.0):
        super(ContrastiveCenterLoss, self).__init__()
        self.dim_hidden = dim_hidden
        self.num_classes = num_classes
        self.lambda_c = lambda_c
        self.centers = nn.Parameter(torch.randn(num_classes, dim_hidden))
        self.device = device
        self.data = {'intra_distances': [], 'inter_distances': [], 'loss': []}
        
    def forward(self, y, hidden):
        batch_size = hidden.size()[0]
        expanded_centers = self.centers.expand(batch_size, -1, -1)
        expanded_centers = expanded_centers.to(self.device)
        hidden = hidden.view(hidden.size(0), -1)
        expanded_hidden = hidden.expand(self.num_classes, -1, -1).transpose(1, 0)
        distance_centers = (expanded_hidden - expanded_centers).pow(2).sum(dim=-1)
        distances_same = distance_centers.gather(1, y.unsqueeze(1))
        intra_distances = distances_same.sum()
        inter_distances = distance_centers.sum().sub(intra_distances)
        epsilon = 1e-6
        loss = (self.lambda_c / 2.0 / batch_size) * intra_distances / \
               (inter_distances + epsilon) / 0.1
        # save data
        self.data['intra_distances'].append(intra_distances.data)
        self.data['inter_distances'].append(inter_distances.data)
        self.data['loss'].append(loss.data)
        return loss

In [6]:
feature_extract = False # fine-tuning
num_classes = 200

resnet18_model_fine_tuning, input_size = initialize_resnet_model(num_classes=num_classes, feature_extract=feature_extract, use_pretrained=True)
resnet18_model_fine_tuning = resnet18_model_fine_tuning.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
# fine-tuning we will be updating all parameters. However, if we are
# doing feature extract method, we will only update the parameters
# that we have just initialized, i.e. the parameters with requires_grad
# is True.
params_to_update = resnet18_model_fine_tuning.parameters()
if feature_extract:
  params_to_update = []
  for name,param in resnet18_model_fine_tuning.named_parameters():
    if param.requires_grad == True:
      params_to_update.append(param)


# Losses
CE_loss = nn.CrossEntropyLoss()                       # <--- Cross Entropy Loss    
center_loss = ContrastiveCenterLoss(dim_hidden=512,   # <--- Contrastive Center Loss
                                    num_classes=num_classes,
                                    device=device,
                                    lambda_c=1.0)
CE_loss, center_loss = CE_loss, center_loss
criterion = [CE_loss, center_loss]

# Optimizers & Scheduler
optimizer_nn = optim.Adam(resnet18_model_fine_tuning.parameters(), lr=1e-6, betas=(0.9, 0.98), eps=1e-09)
optimizer_c = optim.Adagrad(center_loss.parameters(), lr=0.001)         # <--- Optimizer for Contrastive Center Loss
optimizer = [optimizer_nn, optimizer_c]

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_nn, step_size=7, gamma=0.1)

resnet18_model_fine_tuning_trainer = Trainer(resnet18_model_fine_tuning, device, input_size, criterion, 
                                             optimizer, exp_lr_scheduler, batch_size=64, num_epochs=30, num_workers=2, 
                                             dataset_dir='./drive/MyDrive/VoxCeleb1')


In [ ]:
resnet18_model_fine_tuning_trainer.train_model_ccl('train')

Epoch 0/19
----------
train Loss: 5.4323 Acc: 0.0039
val Loss: 5.4447 Acc: 0.0028

Epoch 1/19
----------
train Loss: 5.3524 Acc: 0.0059
val Loss: 5.4227 Acc: 0.0046

Epoch 2/19
----------
train Loss: 5.2788 Acc: 0.0110
val Loss: 5.3978 Acc: 0.0055

Epoch 3/19
----------
train Loss: 5.2074 Acc: 0.0250
val Loss: 5.3809 Acc: 0.0101

Epoch 4/19
----------
train Loss: 5.1386 Acc: 0.0425
val Loss: 5.3595 Acc: 0.0129

Epoch 5/19
----------
train Loss: 5.0704 Acc: 0.0580
val Loss: 5.3466 Acc: 0.0148

Epoch 6/19
----------
train Loss: 5.0014 Acc: 0.0688
val Loss: 5.3285 Acc: 0.0175

Epoch 7/19
----------
train Loss: 4.9577 Acc: 0.0731
val Loss: 5.3294 Acc: 0.0175

Epoch 8/19
----------
train Loss: 4.9522 Acc: 0.0740
val Loss: 5.3293 Acc: 0.0194

Epoch 9/19
----------
train Loss: 4.9438 Acc: 0.0744
val Loss: 5.3226 Acc: 0.0175

Epoch 10/19
----------
train Loss: 4.9368 Acc: 0.0746
val Loss: 5.3254 Acc: 0.0175

Epoch 11/19
----------
